In [52]:
import openpyxl
from openpyxl import load_workbook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

In [53]:
df = pd.read_excel("data1.xlsx", sheet_name=None, header=0)

In [54]:
all_df = {}
for i, d in enumerate(df.items()):
    name, data = d
    all_df[name] = data

In [55]:
data = all_df['1K-01']

In [56]:
data = data[data['DEPTH'] == data["DEPTH"]]   # drop NaN value row

In [57]:
data = data.drop(["Core_Facies"], axis=1)

In [58]:
data['Log_Facies'].apply(lambda x: str(x)).describe()

count     1025
unique       3
top        1.0
freq       530
Name: Log_Facies, dtype: object

In [59]:
comb = [(1, 1), (1, 2), (1, 3), (2, 1), (2, 2), (2, 3)]
def label_facies(row):
    f = row['Fluid']
    lf = row['Log_Facies']
    return comb.index((f, lf))
data['MixLabel'] = data.apply(label_facies, axis=1)

## L R

In [60]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [61]:
X = data[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
ss = StandardScaler()
X = ss.fit_transform(X)

In [62]:
Y = data['Fluid']
#Y = data['MixLabel']

In [63]:
clf = LogisticRegression()

In [64]:
clf.fit(X, Y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [65]:
Y_pred = clf.predict(X)

In [66]:
from sklearn.metrics import confusion_matrix 

In [67]:
results = confusion_matrix(Y, Y_pred)
results

array([[320,   1],
       [ 10, 694]], dtype=int64)

In [68]:
from sklearn.metrics import classification_report
report = classification_report(Y, Y_pred)
print(report)

             precision    recall  f1-score   support

          1       0.97      1.00      0.98       321
          2       1.00      0.99      0.99       704

avg / total       0.99      0.99      0.99      1025



In [69]:
print(clf.intercept_)
print(clf.coef_)

[4.2938129]
[[ 3.51791468  0.49918249 -0.30595433 -0.34224867 -2.30850281 -0.92618712
   0.68215821 -1.18482132 -0.64290689]]


In [70]:
data_test = all_df['1X-02']
data_test = data_test[data_test['DEPTH'] == data_test["DEPTH"]]  
data_test = data_test.drop(["Core_Facies"], axis=1)

data_test['MixLabel'] = data_test.apply(label_facies, axis=1)
X_test = data_test[["DEPTH", "CALI", "DT", "GR", "LLD", "LLS", "MSFL", "NPHI", "RHOB"]]
X_test = ss.fit_transform(X_test)
Y_test = data_test['Fluid']
#Y_test = data_test['MixLabel']
Y_test_pred = clf.predict(X_test)

In [71]:
results = confusion_matrix(Y_test, Y_test_pred)
results

array([[240,   6],
       [ 10, 838]], dtype=int64)

In [72]:
report = classification_report(Y_test, Y_test_pred)
print(report)

             precision    recall  f1-score   support

          1       0.96      0.98      0.97       246
          2       0.99      0.99      0.99       848

avg / total       0.99      0.99      0.99      1094



In [73]:
Y_t = Y_test[Y_test_pred==Y_test]

In [75]:
Y_t.shape[0]/Y_test.shape[0]

0.9853747714808044

In [76]:
Y_t.shape[0]

1078

In [77]:
Y_test.shape[0]

1094